In [1]:
# Para cargar la data
import polars as pl

In [14]:
# Cargamos metadata: info de las empresas
metadata = pl.read_csv('../../data/google/metadata.csv')

In [15]:
# Seleccionamos las columnas que vamos a utilizar y quedarnos para produccion.
metadata = metadata.select(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'misc', 'state', 'relative_results', 'url'])

In [12]:
# Lo utilizaremos como referencia
business = pl.read_parquet('../../data/yelp/production/business.parquet')

**ELIMINAMOS DUPLICADOS**

In [25]:
# Consideramos duplicados a aquellos con mismo gmap_id y/o aquellos en el mismo lugar y mismo nombre.
metadata = metadata.filter(pl.col('gmap_id').is_unique())
metadata = metadata.unique(subset=['latitude', 'longitude', 'name'])

**NAME**

In [17]:
# Transformamos a lowercase
metadata = metadata.with_columns(pl.col('name').str.to_lowercase())

In [19]:
# Cuantos duplicados hay de name
metadata['name'].is_duplicated().sum()

846198

**ADDRESS**

In [43]:
# Ejemplo
metadata['address'][1] # Se divide en nombre, calle, ciudad, estado codigo postal

'Vons Chicken, 12740 La Mirada Blvd, La Mirada, CA 90638'

In [44]:
def split_address(address):
    try:
        address = address.lower()
        data = [x.strip() for x in address.split(',')]
        name = data[0]
        street = data[1]
        city = data[2]
        state, postal_code = [x.strip() for x in data[3].split()]
        return street, city, state, postal_code
    except:
        return None

In [48]:
# temp es un dataframe que tiene como columna address cuyo valores son listas con la siguiente info: [street, city, state, postal_code]
temp = metadata.select(pl.col('address').apply(split_address))

In [65]:
# Creamos en metadata las columnas correspondientes usando temp
metadata = metadata.with_columns(temp['address'].list.get(0).alias('street'))
metadata = metadata.with_columns(temp['address'].list.get(1).alias('city'))
metadata = metadata.with_columns(temp['address'].list.get(2).alias('state'))
metadata = metadata.with_columns(temp['address'].list.get(3).alias('postal_code'))

**GMAP_ID**

In [68]:
# Cantidad de nulos
metadata['gmap_id'].null_count()

0

In [67]:
# Cantidad de duplicados
metadata['gmap_id'].is_duplicated().sum()

0

**DESCRIPTION**

In [69]:
# Cantidad de nulos
metadata['description'].null_count()

2744777

In [70]:
# La eliminamos para reducir espacio
metadata = metadata.drop('description')

**LATITUDE**

In [72]:
# Debe ser valor positivo
(metadata['latitude'] < 0).sum()

13

In [75]:
# Convierto a valores positivos
metadata = metadata.with_columns(pl.col('latitude').abs())
(metadata['latitude'] < 0).sum()

0

In [76]:
# Numero de duplicados
metadata['latitude'].is_duplicated().sum()

294193

In [104]:
# Numero de valores faltantes en latitude
metadata['latitude'].null_count()

0

**LONGITUDE**

In [77]:
# Debe ser valor negativo
(metadata['longitude'] > 0).sum()

88

In [79]:
# Convierto a valores negativos
metadata = metadata.with_columns(pl.col('longitude').abs() * (-1))
(metadata['longitude'] > 0).sum()

0

In [80]:
# Numero de duplicados
metadata['longitude'].is_duplicated().sum()

337982

In [82]:
# Numero de duplicados con la misma longitude y latitude
metadata.shape[0] - metadata.unique(subset=['latitude', 'longitude']).shape[0]

133225

In [105]:
# Numero de valores faltantes en longitude
metadata['longitude'].null_count()

0

**CATEGORY**

In [96]:
# Convierte la lista de valores a una string separada por coma.
def from_list_to_str(value, sep=','):
    try:
        return sep.join([x.lower().strip() for x in eval(value)])
    except:
        return None
            

In [97]:
# Ejemplo
example = metadata['category'][3]
print('EXAMPLE:', example)
print('OUTPUT:', from_list_to_str(example))

EXAMPLE: ['Greeting card shop', 'Service establishment']
OUTPUT: greeting card shop,service establishment


In [100]:
# Aplicamos la funcion
metadata = metadata.with_columns(pl.col('category').apply(from_list_to_str))

In [99]:
# Cantidad de valores unicos.
metadata['category'].unique().shape[0]

356039

In [102]:
# Cantidad de valores unicos.
metadata['category'].null_count()

17181

**AVG_RATING**

In [106]:
# Debe ser un valor positivo y estar entre 1 y 5.
metadata['avg_rating'].min(), metadata['avg_rating'].max()

(1.0, 5.0)

**NUM OF REVIEWS**

In [107]:
# Debe ser un valor positivo
metadata['num_of_reviews'].min(), metadata['num_of_reviews'].max()

(1, 9998)

**PRICE**

In [113]:
# Cantidad de nulos
metadata['price'].null_count()

2723877

In [115]:
# La eliminamos para ahorrar espacio
metadata = metadata.drop('price')

**HOURS**

In [122]:
# Cantidad de nulos
metadata['hours'].null_count()

779383

In [119]:
# Ejemplo
eval(metadata['hours'][0])

[['Friday', '8AM–6PM'],
 ['Saturday', '8AM–12PM'],
 ['Sunday', 'Closed'],
 ['Monday', '8AM–6PM'],
 ['Tuesday', '8AM–6PM'],
 ['Wednesday', '8AM–12PM'],
 ['Thursday', '8AM–6PM']]

La mantendremos asi para simplificar el dataset

**MISC**

In [121]:
# Cantidad de nulos
metadata['misc'].null_count()

682866

In [120]:
# Ejemplo
eval(metadata['misc'][0])

{'Service options': ['In-store shopping', 'Same-day delivery'],
 'Health & safety': ['Mask required',
  'Staff required to disinfect surfaces between visits'],
 'Accessibility': ['Wheelchair accessible entrance'],
 'Planning': ['Quick visit']}

Lo mismo que el anterior, para evitar sobrecargar en columnas, lo dejamos en formato struct.

**STATE CITY POSTAL_CODE**

In [128]:
# Cantidad de unicos en state
metadata['state'].unique().shape[0]

3143

In [129]:
# Cantidad de unicos en city
metadata['city'].unique().shape[0]

64779

In [130]:
# Cantidad de unicos en postal_code
metadata['postal_code'].unique().shape[0]

35607

**RELATIVE_RESULTS**

In [132]:
# Es una lista de gmap_id con la que la empresa esta relacionada pareciera ser
metadata['relative_results'][0]

"['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x88f16c32716531c1:0x5f19bdaa5044e4fa', '0x88f16e6e3f4a21df:0xcf495da9bb4d89ea']"

In [133]:
# Cantidad de nulos
metadata['relative_results'].null_count()

291292

**COMPANY_INDEX**

In [138]:
# Creamos una nueva columna id porque la de gmap_id es muy compleja
metadata = metadata.with_row_count(offset=1, name='company_index')

In [139]:
# Almacenamos la data
metadata.write_parquet('../../data/google/production/metadata.parquet')